In [97]:

import os
import json
from google.cloud import storage
import pandas as pd
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('punkt')
nltk.download('all')
from nltk.tokenize import word_tokenize
# cleaning up from stopwords
from nltk.corpus import stopwords

# lemmatizing the words
from nltk.stem import WordNetLemmatizer

# vectorizing the tokens
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /Users/dima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/dima/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/dima/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/dima/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/dima/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/dima/nltk_data...


In [5]:
# #bucket name
bucket_name = 'flavor-forecast-raw-data'

# # initialize a GCS client
storage_client = storage.Client()

# # get the right bucket
bucket = storage_client.bucket(bucket_name)

In [6]:
# storage_filename = "yelp-dataset/yelp_academic_dataset_business.json"
# local_filename = "yelp_business_raw.json"

# blob = bucket.blob(storage_filename)
# blob.download_to_filename(local_filename)


# # # creating a json file path to take the input form
# # json_file_path = "yelp_business_raw.json"

# # #reading the json file
# # df_business_raw = pd.read_json(json_file_path, lines=True)

# # # creating a destination path
# # csv_file_path = "yelp_business_raw.csv"

# # #converting the output into a csv file
# # df_business_raw.to_csv(csv_file_path, index=False)

In [7]:
# using storage filename for reviews raw dataset
storage_filename_review = "yelp-dataset/yelp_academic_dataset_review.json"
local_filename_review = "yelp_dataset_review_raw.json"

# downloading the dataset
blob = bucket.blob(storage_filename_review)
blob.download_to_filename(local_filename_review)

In [8]:
# creating a json file path to take the input form
json_file_path_review = "yelp_dataset_review_raw.json"

#reading the json file
df_review_raw = pd.read_json(json_file_path_review, lines=True)

# creating a destination path
csv_file_path_review = "yelp_review_raw.csv"

#converting the output into a csv file
df_review_raw.to_csv(csv_file_path_review, index=False)

In [65]:
df_review_raw.shape

(6990280, 9)

In [17]:
# load restaurants filter file
res_ids = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants.csv")

In [20]:
all_ids = df_review_raw.business_id

print(f'Number of total unique business IDs in the entire dataset: {all_ids.nunique()}')
print(f'Number of total unique restaurant IDs: {res_ids.business_id.nunique()}')


Number of total unique business IDs in the entire dataset: 150346
Number of total unique restaurant IDs: 52268


In [26]:
df_review_none_filt = df_review_raw[df_review_raw['business_id'].isin(all_ids)]
df_review_res_filt = df_review_raw[df_review_raw['business_id'].isin(res_ids.business_id)]

print(f'Shape of the dataset by including all businesses: {df_review_none_filt.shape}')
print(f'Shape of the dataset by applying restaurant-only filter: {df_review_res_filt.shape}')


Shape of the dataset by including all businesses: (6990280, 9)
Shape of the dataset by applying restaurant-only filter: (4724471, 9)


In [66]:
df_review_res_filt.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
5,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31


In [69]:
df_review_fil_pos = df_review_res_filt[df_review_res_filt.stars >= 4]
df_review_fil_neg = df_review_res_filt[df_review_res_filt.stars < 4]

print('Positive reviews are those with 4 and 5 star rating. Negative reviews are considered reviews below stars')
print(f'Number of positive reviews: {df_review_fil_pos.shape[0]}')
print(f'Number of negative reviews: {df_review_fil_neg.shape[0]}')

Positive reviews are those with 4 and 5 star rating. Negative reviews are considered reviews below stars
Number of positive reviews: 3209692
Number of negative reviews: 1514779


In [74]:
reviews = df_review_fil_pos.text
reviews.head()

3     Wow!  Yummy, different,  delicious.   Our favo...
4     Cute interior and owner (?) gave us tour of up...
7     Amazingly amazing wings and homemade bleu chee...
11    Locals recommended Milktooth, and it's an amaz...
12    Love going here for happy hour or dinner!  Gre...
Name: text, dtype: object

In [75]:
def strip_lower(reviews):
    # converting the reviews column into a list before stripping
    reviews_list = list(reviews)
    reviews_list_strip_lower = [review.strip().lower() for review in reviews_list]

    # lowering the letters in the reviews
    for review in reviews_list_strip_lower:
        for punctuation in string.punctuation:
            review = review.replace(punctuation,'')

    return reviews_list_strip_lower

reviews_list_strip_lower = strip_lower(reviews)

In [79]:
df_review_pos = pd.DataFrame(reviews_list_strip_lower,columns=['reviews'])
print(f'DF # of reviews check after strip-lower-punctuation preproc: {df_review_pos.shape[0]==df_review_fil_pos.shape[0]}')

DF # of reviews check after strip-lower-punctuation preproc: True


In [80]:
# sampling the
df_pos_sample = df_review_pos.sample(1000000)
df_pos_sample.head()

,reviews
1176091,i'm originally from ny and i thought ny had th...
1468871,i can't believe i'm so late on this..... ramen...
2423702,we are from memphis and we try to compare vari...
2493962,i am obsessed with this little place. it's a s...
2365532,first time going to this place and no doubt ab...


In [81]:
df_pos_sample['tokenizer'] = df_pos_sample["reviews"].apply(word_tokenize)

In [83]:
df_pos_sample.head()

,reviews,tokenizer
1176091,i'm originally from ny and i thought ny had th...,"[i, 'm, originally, from, ny, and, i, thought,..."
1468871,i can't believe i'm so late on this..... ramen...,"[i, ca, n't, believe, i, 'm, so, late, on, thi..."
2423702,we are from memphis and we try to compare vari...,"[we, are, from, memphis, and, we, try, to, com..."
2493962,i am obsessed with this little place. it's a s...,"[i, am, obsessed, with, this, little, place, ...."
2365532,first time going to this place and no doubt ab...,"[first, time, going, to, this, place, and, no,..."


In [84]:
def clean_stopwords(df_pos_sample):
    stop_words = set(stopwords.words('english'))
    df_sample_tokens = df_res_sample.tokenizer

    df_tokens_clean = []
    for list in df_sample_tokens:
        for w in list:
            tokens_clean = [w for w in list if not w in stop_words]
        df_tokens_clean.append(tokens_clean)

    return df_tokens_clean

df_pos_sample['tokenizer_clean'] = clean_stopwords(df_pos_sample)


In [85]:
df_pos_sample.head()

,reviews,tokenizer,tokenizer_clean
1176091,i'm originally from ny and i thought ny had th...,"[i, 'm, originally, from, ny, and, i, thought,...","[two, years, ,, drove, past, evviva, ,, admiri..."
1468871,i can't believe i'm so late on this..... ramen...,"[i, ca, n't, believe, i, 'm, so, late, on, thi...","[great, laid, back, bar, hidden, gem, davis, i..."
2423702,we are from memphis and we try to compare vari...,"[we, are, from, memphis, and, we, try, to, com...","[perfection, roll, (, wrap, ), ., 've, never, ..."
2493962,i am obsessed with this little place. it's a s...,"[i, am, obsessed, with, this, little, place, ....","[husband, came, celebrate, first, day, new, jo..."
2365532,first time going to this place and no doubt ab...,"[first, time, going, to, this, place, and, no,...","[customer, services, n't, best, ., first, time..."


In [88]:
def lemmatize_verb_nount(lemmatize_input):
    # lemmatize the verb
    sample_lemmatize_v = []
    for review in lemmatize_input:
        review_lemmatized_verb = [
        WordNetLemmatizer().lemmatize(word,pos='v')
        for word in review
        ]
        sample_lemmatize_v.append(review_lemmatized_verb)

    #lemmatizing the nouns
    sample_lemmatize_v_n = []
    for review in sample_lemmatize_v:
        review_lemmatized_noun = [
        WordNetLemmatizer().lemmatize(word,pos='v')
        for word in review
        ]
        sample_lemmatize_v_n.append(review_lemmatized_noun)

    return sample_lemmatize_v_n

In [89]:
df_pos_sample['tokens_lemm'] = lemmatize_verb_nount(df_pos_sample.tokenizer_clean)

In [90]:
df_pos_sample.head()

,reviews,tokenizer,tokenizer_clean,tokens_lemm
1176091,i'm originally from ny and i thought ny had th...,"[i, 'm, originally, from, ny, and, i, thought,...","[two, years, ,, drove, past, evviva, ,, admiri...","[two, years, ,, drive, past, evviva, ,, admire..."
1468871,i can't believe i'm so late on this..... ramen...,"[i, ca, n't, believe, i, 'm, so, late, on, thi...","[great, laid, back, bar, hidden, gem, davis, i...","[great, lay, back, bar, hide, gem, davis, isla..."
2423702,we are from memphis and we try to compare vari...,"[we, are, from, memphis, and, we, try, to, com...","[perfection, roll, (, wrap, ), ., 've, never, ...","[perfection, roll, (, wrap, ), ., 've, never, ..."
2493962,i am obsessed with this little place. it's a s...,"[i, am, obsessed, with, this, little, place, ....","[husband, came, celebrate, first, day, new, jo...","[husband, come, celebrate, first, day, new, jo..."
2365532,first time going to this place and no doubt ab...,"[first, time, going, to, this, place, and, no,...","[customer, services, n't, best, ., first, time...","[customer, service, n't, best, ., first, time,..."


In [93]:
def stringize_tokens(df_pos_sample):
    vector_input = []
    for review in df_pos_sample:
        review_string = " ".join(review)
        vector_input.append(review_string)

    return vector_input

In [98]:
reviews_preproc = stringize_tokens(df_pos_sample.tokens_lemm)
len(reviews_preproc)

1000000

In [103]:
tf_idf_vectorizer = TfidfVectorizer(
                    ngram_range=(2,4),
                    min_df=0.001,
                    max_df=0.1
)

weighted_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(reviews_preproc).toarray(),
                              columns=tf_idf_vectorizer.get_feature_names_out())

In [104]:
weighted_words

,10 10,10 15,10 15 minutes,10 min,10 mins,10 minutes,10 minutes later,10 would,10 years,11 30,...,year old,years ago,years never,years since,yelp review,young lady,young man,yum yum,yummy food,zero star
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
sum_tfidf = weighted_words.sum(axis=0)
sum_tfidf_sorted = sum_tfidf.sort_values(ascending=False)

In [108]:
sum_tfidf_sorted.head(30)

go back             11288.957656
come back           10400.314226
highly recommend     8745.081650
great food           8155.123897
food good            8151.353881
first time           7938.396260
really good          7462.664834
great service        6903.012994
food great           6890.014342
good food            6527.470439
ve ever              6435.214912
love place           6381.835739
customer service     6277.080832
great place          6148.266669
pretty good          5861.015161
service great        5833.884494
happy hour           5426.140196
next time            5374.608562
new orleans          5234.411122
taste like           5173.643430
staff friendly       4912.829493
food service         4905.661689
mac cheese           4749.275234
make sure            4743.900528
one best             4607.914958
every time           4572.232429
ice cream            4491.296963
good service         4454.969550
service good         4352.559152
definitely back      4276.971660
dtype: flo